In [1]:
from genericpath import isfile
import json
import os

if __name__ == '__main__':
    current_dir = os.path.dirname(os.path.abspath("/home/song/Desktop/song/lightweight_3DSSG/size_test"))
    os.sys.path.append(current_dir)
    os.sys.path.append(os.path.join(current_dir, './src'))
    #os.sys.path.append('./pytorch_geometric/torch_geometric')
   # os.sys.path.append('./src')
from src.model.model import MMGNet
from src.utils.config import Config
from utils import util
import torch
import argparse
import torch.nn.utils.prune as prune

In [2]:
def load_config():
    # load config file
    config = Config("/home/song/Desktop/song/lightweight_3DSSG/config/SGGpoint.json")
    #print(config)
    config.MODE = 'train'
    config.exp = 'test'
    #config.pruning.st_pruning_ratio = 0.5
    #config.pruning_method = "st"
    
    return config
config = load_config()
os.environ["CUDA_LAUNCH_BLOCKING"] = "0"
util.set_random_seed(config.SEED)
config.MODEL.use_pretrain ='x'
# if config.VERBOSE:
#     print(config)
model = MMGNet(config)
#model.gcn_pruning()


model name : SGGpoint
num of data: 3845
num of data: 548
path: x/backbone_best.pth result: False
path: x/mlp_3d_best.pth result: False
path: x/edge_mlp_best.pth result: False
path: x/edge_gcn_best.pth result: False
path: x/obj_mlp_best.pth result: False
path: x/rel_mlp_best.pth result: False
path: x/obj_classifier_best.pth result: False
path: x/rel_classifier_best.pth result: False
	model loading failed!

load pretrain model: x


In [3]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
def get_submodule_parameters(model):
    submodule_params = {}
    for name, module in model.named_children():
        submodule_params[name] = count_parameters(module)
    return submodule_params

In [4]:
submodule_params = get_submodule_parameters(model.model.edge_gcn)

print("각 서브모듈의 파라미터 수:")
for name, params in submodule_params.items():
    print(f"{name}: {params:,}")
# 전체 파라미터 수 계산 및 출력
total_params = count_parameters(model.model.edge_gcn)
print(f"총 파라미터 수: {total_params:,}")

각 서브모듈의 파라미터 수:
node_GConv1: 131,328
node_GConv2: 131,584
edge_MLP1: 131,328
edge_MLP2: 131,584
edge_attentionND: 131,328
node_attentionND: 131,328
node_indicator_reduction: 131,328
총 파라미터 수: 919,808


In [7]:
model.model.edge_gcn

EdgeGCN(
  (node_GConv1): GCNConv(512, 256)
  (node_GConv2): GCNConv(256, 512)
  (edge_MLP1): Sequential(
    (0): Conv1d(512, 256, kernel_size=(1,), stride=(1,))
    (1): ReLU()
  )
  (edge_MLP2): Sequential(
    (0): Conv1d(256, 512, kernel_size=(1,), stride=(1,))
    (1): ReLU()
  )
  (edge_attentionND): Linear(in_features=512, out_features=256, bias=True)
  (node_attentionND): Linear(in_features=512, out_features=256, bias=True)
  (node_indicator_reduction): Linear(in_features=512, out_features=256, bias=True)
)

total_params: 10677128


In [72]:
model.model.mmg

MMG(
  (self_attn): ModuleList(
    (0): MultiHeadAttention(
      (attention): ScaledDotProductAttention(
        (fc_q): Linear(in_features=512, out_features=512, bias=True)
        (fc_k): Linear(in_features=512, out_features=512, bias=True)
        (fc_v): Linear(in_features=512, out_features=512, bias=True)
        (fc_o): Linear(in_features=512, out_features=512, bias=True)
      )
      (dropout): Dropout(p=0.1, inplace=False)
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    )
    (1): MultiHeadAttention(
      (attention): ScaledDotProductAttention(
        (fc_q): Linear(in_features=512, out_features=512, bias=True)
        (fc_k): Linear(in_features=512, out_features=512, bias=True)
        (fc_v): Linear(in_features=512, out_features=512, bias=True)
        (fc_o): Linear(in_features=512, out_features=512, bias=True)
      )
      (dropout): Dropout(p=0.1, inplace=False)
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  

: 

In [54]:
model.apply_pruning("gnn")

gnn pruning start
gnn: mmg pruning:0.5 start!
gnn pruning success!


In [ ]:
for name, param in model.model.named_parameters():
    if param.requires_grad:
        num_params = param.numel()
        if name.endswith('weight'):
            non_zero_params = torch.count_nonzero(param).item()
        else:
            non_zero_params = num_params
        print(f'name: {name}, num_params: {num_params}, non_zero_params: {non_zero_params}')

In [43]:
tripletModel = model.model.gcn.gconvs[0]
layers = []
for m in tripletModel.named_modules():
    print(m)
#     if isinstance(m, torch.nn.Linear):
#         layers.append(m)
#     elif isinstance(m, torch.nn.modules.conv._ConvNd):
#         layers.append(m)
# print(layers)

('', TripletGCN())
('aggr_module', SumAggregation())
('nn1', Sequential(
  (0): Linear(in_features=1280, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=1280, bias=True)
  (3): ReLU()
))
('nn1.0', Linear(in_features=1280, out_features=512, bias=True))
('nn1.1', ReLU())
('nn1.2', Linear(in_features=512, out_features=1280, bias=True))
('nn1.3', ReLU())
('nn2', Sequential(
  (0): Linear(in_features=512, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=512, bias=True)
))
('nn2.0', Linear(in_features=512, out_features=512, bias=True))
('nn2.1', ReLU())
('nn2.2', Linear(in_features=512, out_features=512, bias=True))


In [31]:
model.model.gcn

TripletGCNModel(
  (gconvs): ModuleList(
    (0): TripletGCN()
    (1): TripletGCN()
  )
)

In [ ]:
for name, module in model.model.named_modules():
    if isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=0.5)
        prune.remove(module, 'weight')
    elif isinstance(module, torch.nn.Conv1d):
        prune.l1_unstructured(module, name='weight', amount=0.5)
        prune.remove(module, 'weight')

In [22]:
model.model.mmg.self_attn

AttributeError: 'SGFN' object has no attribute 'mmg'

In [11]:
model.model.mmg.self_attn[0].attention.fc_o

Linear(in_features=512, out_features=512, bias=True)

In [20]:
import torch_pruning as tp

In [29]:
def apply_pruning(self, apply_part):
        if apply_part == "encoder":
            encoders = ['obj_encoder', 'rel_encoder_2d', 'rel_encoder_3d']
            for encoder_name in encoders:
                print(f"encoder: {encoder_name} pruning:{self.pruning_ratio} start!")
                for name, module in getattr(self.model, encoder_name).named_modules():
                    if isinstance(module, torch.nn.Conv1d):
                        prune.l1_unstructured(module, name='weight', amount=self.pruning_ratio)
                        prune.remove(module, 'weight')
        elif apply_part == "gnn":
            print("gnn pruning start")
            gnn_name = 'mmg'
            print(f"gnn: {gnn_name} pruning:{self.pruning_ratio} start!")
            for name, module in getattr(self.model, gnn_name).named_modules():
                if isinstance(module, torch.nn.Linear):
                    prune.l1_unstructured(module, name='weight', amount=self.pruning_ratio)
                    prune.remove(module, 'weight')
        elif apply_part == "classifier":
            print(f"classifier: {apply_part} pruning:{self.pruning_ratio} start!")
            classifiers = ['obj_predictor_3d', 'rel_predictor_3d', 'obj_predictor_2d', 'rel_predictor_2d']
            for predicator in classifiers:
                for name, module in getattr(self.model, predicator).named_modules():
                    if isinstance(module, torch.nn.Linear):
                        prune.l1_unstructured(module, name='weight', amount=self.pruning_ratio)
                        prune.remove(module, 'weight')
        elif apply_part == 'all':
            print(f"ALL Pruning start!")
            for name, module in self.model.named_modules():
                if isinstance(module, torch.nn.Linear):
                    prune.l1_unstructured(module, name='weight', amount=self.pruning_ratio)
                    prune.remove(module, 'weight')
                elif isinstance(module, torch.nn.Conv1d):
                    prune.l1_unstructured(module, name='weight', amount=self.pruning_ratio)
                    prune.remove(module, 'weight')
        else:
            print("pruning error!")
        print(f"{apply_part} pruning success!")

Head #0


AttributeError: 'MMG_student' object has no attribute 'head_dim'

In [60]:
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
import torch.optim as optim

# Seed 설정
def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_random_seed(2020)

# 간단한 모델 정의
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 3)
        self.fc1 = nn.Linear(16 * 3, 1)  # Conv1d 출력 크기에 맞게 Linear 층 수정

    def forward(self, x):
        x = self.conv1(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

model = SimpleModel()

# Pruning 적용
for name, module in model.named_modules():
    if isinstance(module, nn.Conv1d):
        prune.l1_unstructured(module, name='weight', amount=0.3)
        prune.remove(module, 'weight')

# 학습 설정
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Dummy 데이터
inputs = torch.randn(10, 1, 5)
targets = torch.randn(10, 1)

# 학습
for epoch in range(5):
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# 가지치기된 파라미터 확인
for name, module in model.named_modules():
    if isinstance(module, nn.Conv1d):
        print(f'{name} weight after pruning and training:')
        print(module.weight)


Epoch 1, Loss: 0.862275242805481
Epoch 2, Loss: 0.6179301738739014
Epoch 3, Loss: 0.45785441994667053
Epoch 4, Loss: 0.3521365225315094
Epoch 5, Loss: 0.28182896971702576
conv1 weight after pruning and training:
Parameter containing:
tensor([[[ 4.7807e-03, -4.5514e-01,  6.1754e-04]],

        [[-4.4016e-01, -9.2592e-04, -2.4921e-01]],

        [[ 1.5271e-03,  3.5766e-01, -2.8726e-01]],

        [[ 1.4692e-01,  4.4684e-03,  3.4911e-01]],

        [[ 3.2163e-01,  4.3871e-04,  4.4892e-01]],

        [[-4.4697e-03,  2.4309e-01, -5.4016e-01]],

        [[-4.4790e-01,  4.4176e-01, -5.0021e-01]],

        [[-1.7102e-03,  3.6067e-01, -1.3293e-01]],

        [[-2.9553e-03, -3.6057e-03, -5.1928e-01]],

        [[ 2.9437e-01, -2.0267e-01, -1.5559e-03]],

        [[ 2.3055e-01,  5.6920e-01, -2.5067e-03]],

        [[-4.3774e-01,  2.5613e-01, -4.2329e-01]],

        [[ 3.3195e-01,  9.8718e-04,  4.0817e-01]],

        [[-7.6961e-04, -5.4603e-01,  4.5536e-01]],

        [[-3.9488e-01,  4.3384e-01,  3